In [ ]:
%pip install openai

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema.messages import HumanMessage
import os
from dotenv import load_dotenv

load_dotenv()  # .env 파일 불러오기

llm = ChatOpenAI(model="gpt-4o-mini")

response = llm.invoke([HumanMessage(content="LangSmith에서 이 대화 추적되나요?")])
print(response.content)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema.messages import HumanMessage
import os
from dotenv import load_dotenv
from langchain_teddynote.models import MultiModal
from langchain_teddynote.messages import stream_response

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-4o",  # 모델명
)

# 멀티모달 객체 생성
multimodal_llm = MultiModal(llm)

# 샘플 이미지 주소(웹사이트로 부터 바로 인식)
IMAGE_URL = "https://t3.ftcdn.net/jpg/03/77/33/96/360_F_377339633_Rtv9I77sSmSNcev8bEcnVxTHrXB4nRJ5.jpg"

# 이미지 파일로 부터 질의
answer = multimodal_llm.stream(IMAGE_URL)
# 스트리밍 방식으로 각 토큰을 출력합니다. (실시간 출력)
stream_response(answer)


In [ ]:
from langchain_openai import ChatOpenAI
from IPython.display import Image, display
import requests
import base64

# 이미지 URL
IMAGE_URL = "https://t3.ftcdn.net/jpg/03/77/33/96/360_F_377339633_Rtv9I77sSmSNcev8bEcnVxTHrXB4nRJ5.jpg"

# 이미지 → base64 인코딩
def encode_image_from_url(url):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception("이미지 다운로드 실패")
    image_content = response.content
    ext = url.lower().split('.')[-1]
    mime_type = f"image/{'jpeg' if ext in ['jpg', 'jpeg'] else 'png'}"
    encoded = base64.b64encode(image_content).decode("utf-8")
    return f"data:{mime_type};base64,{encoded}"

# LangChain 모델 생성 (GPT-4o)
llm = ChatOpenAI(
    temperature=0.1,
    max_tokens=2048,
    model_name="gpt-4o"
)

# 메시지 구성
encoded_image = encode_image_from_url(IMAGE_URL)

messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that explains images in detail.",
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Please analyze and explain this image.",
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": encoded_image,
                },
            },
        ],
    },
]

# 이미지 표시 (선택)
display(Image(url=IMAGE_URL))

# 스트리밍 응답 출력
for chunk in llm.stream(messages):
    print(chunk.content, end="", flush=True)
